In [ ]:
import os
import shutil
from collections import Counter
from collections import namedtuple

import numpy as np
import matplotlib.pyplot as plt
import cv2

from sdcdup.utils import get_project_root
from sdcdup.utils import get_tile
from sdcdup.utils import load_duplicate_truth
from sdcdup.utils import ImgMod
from sdcdup.features import SDCImageContainer
from sdcdup.features import load_image_overlap_properties
from sdcdup.visualization import get_ticks
from sdcdup.visualization import draw_tile_number
from sdcdup.visualization import draw_tile_bbox
from sdcdup.visualization import show_image_pair

%load_ext dotenv
%dotenv
%matplotlib inline
%reload_ext autoreload
%autoreload 2

RED = (244, 67, 54)  #F44336
GREEN = (76, 175, 80)  #4CAF50
BLUE = (3, 169, 244)  #03A9F4

SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 16
BIGGEST_SIZE = 20
plt.rc('font', size=BIGGER_SIZE)         # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGEST_SIZE)  # fontsize of the figure title

project_root = get_project_root()
train_image_dir = os.path.join(project_root, os.getenv('RAW_DATA_DIR'), 'train_768')
persistent_data_dir = os.path.join(project_root, os.getenv('PERSISTENT_DATA_DIR'))
samples_dir = os.path.join(persistent_data_dir, 'samples')
figures_dir = os.path.join(project_root, 'notebooks', 'figures')

bbox_color_map = {'red': RED, 'blue': BLUE, 'green': GREEN}

ticks = get_ticks()

In [ ]:
def cp2persist(img_id):
    src = os.path.join(train_image_dir, img_id)
    dst = os.path.join(samples_dir, img_id)
    if not os.path.exists(dst):
        shutil.copy2(src, dst)

def get_overlap_pixel_scores(img1_id, img2_id, img1_overlap_tag):
    scores = None
    if (img1_id, img2_id) in overlap_image_maps:
        if img1_overlap_tag in overlap_image_maps[(img1_id, img2_id)]:
            scores = overlap_image_maps[(img1_id, img2_id)][img1_overlap_tag]
    if scores is None:
        Scores = namedtuple('Scores', 'px0 pix')
        px0 = sdcic.gen_px0_scores(img1_id, img2_id, img1_overlap_tag)
        pix = sdcic.gen_pix_scores(img1_id, img2_id, img1_overlap_tag)
        scores = Scores(px0, pix)
    return scores

def plot_image_grid(img_ids, ncols, nrows, filebase=None):
    fig, ax = plt.subplots(nrows, ncols, figsize=(ncols*4, nrows*4))
    for i, img_id in enumerate(img_ids):
        img = cv2.cvtColor(cv2.imread(os.path.join(train_image_dir, img_id)), cv2.COLOR_BGR2RGB)
        ax[i // ncols, i % ncols].imshow(img)
        ax[i // ncols, i % ncols].set_title(img_id)
        ax[i // ncols, i % ncols].set_xticks(ticks)
        ax[i // ncols, i % ncols].set_yticks(ticks)
    plt.tight_layout()
    
    if filebase:
        filename = f'{filebase}_{ncols}_{nrows}.jpg'
        plt.savefig(os.path.join(figures_dir, filename))

In [ ]:
sdcic = SDCImageContainer()
sdcic.preprocess_image_properties()

In [ ]:
n_matching_tiles_list = [9, 6, 4, 3, 2, 1]
overlap_image_maps = load_image_overlap_properties(n_matching_tiles_list)
print(len(overlap_image_maps))

In [ ]:
dup_truth = load_duplicate_truth(filepath=persistent_data_dir, filename='duplicate_truth.txt')
print(len(dup_truth))

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 5
line_type = 8
bbox_thickness = 8
tile_indexes = [0, 1, 2, 3, 4, 5, 6, 7, 8]
img_id = '21d7ea9bf.jpg'

fig, ax = plt.subplots(1, 1, figsize=(4, 4))

img = cv2.cvtColor(cv2.imread(os.path.join(samples_dir, img_id)), cv2.COLOR_BGR2RGB)
for idx in tile_indexes:
    draw_tile_bbox(img, idx, bbox_thickness, RED)
    draw_tile_number(img, idx, color=RED)
    
ax.imshow(img)
ax.set_title(img_id)
ax.set_xticks(ticks)
ax.set_yticks(ticks)

plt.savefig(os.path.join(figures_dir, f'overlap_key.jpg'))

## Check to see how many exact duplicate tiles we have.

In [ ]:
dup_tile_hashes = {}
dup_tile_counts = Counter()
dup_image_samples = []
for img_id, tile_hashes in sdcic.tile_md5hash_grids.items():
    c0 = Counter(tile_hashes)
    for tile_hash, c in c0.items():
        if c == 1:
            continue
        tile_indexes = np.where(tile_hashes == tile_hash)[0]
        
        if tile_hash not in dup_tile_hashes:
            dup_image_samples.append((img_id, tile_hash, tile_indexes))
            dup_tile_hashes[tile_hash] = {}

        for idx in tile_indexes:
            if img_id not in dup_tile_hashes[tile_hash]:
                dup_tile_hashes[tile_hash][img_id] = []
            if idx not in dup_tile_hashes[tile_hash][img_id]:
                dup_tile_hashes[tile_hash][img_id].append(idx)
                dup_tile_counts[tile_hash] += 1

print(dup_tile_counts)

dup_hashes = tuple([key for key in dup_tile_hashes])

for img_id, tile_hashes in sdcic.tile_md5hash_grids.items():
    c0 = Counter(tile_hashes)
    for tile_hash, c in c0.items():
        if c > 1:
            continue
        if tile_hash not in dup_hashes:
            continue
        tile_indexes = np.where(tile_hashes == tile_hash)[0]
        
        for idx in tile_indexes:
            if img_id not in dup_tile_hashes[tile_hash]:
                dup_tile_hashes[tile_hash][img_id] = []
            if idx not in dup_tile_hashes[tile_hash][img_id]:
                dup_tile_hashes[tile_hash][img_id].append(idx)
                dup_tile_counts[tile_hash] += 1
                
print(dup_tile_counts)

dup_image_counts = {key: len(val) for key, val in dup_tile_hashes.items()}
print(dup_image_counts)

bbox_thickness = 8
ncols = 3
nrows = 2
assert ncols * nrows == len(dup_image_samples)

fig, ax = plt.subplots(nrows, ncols, figsize=(15, 10))

for i, (img_id, tile_hash, tile_indexes) in enumerate(dup_image_samples):
    cp2persist(img_id)
    img = cv2.cvtColor(cv2.imread(os.path.join(samples_dir, img_id)), cv2.COLOR_BGR2RGB)
    
    title = f'{img_id} | {tile_hash}'
    
    for ii, idx in enumerate(tile_indexes):
        if ii == 0:
            tile = get_tile(img, idx)
            print(img_id, tile_hash, dup_image_counts[tile_hash], dup_tile_counts[tile_hash])
#             print(tile[0, 0], tile[0, -1])
#             print(tile[-1, 0], tile[-1, -1])

    for idx in tile_indexes:
        draw_tile_bbox(img, idx, bbox_thickness, RED)
        draw_tile_number(img, idx, color=RED)
    
    ax[i // ncols, i % ncols].imshow(img)
    ax[i // ncols, i % ncols].set_title(title)
    ax[i // ncols, i % ncols].set_xticks(ticks)
    ax[i // ncols, i % ncols].set_yticks(ticks)

plt.tight_layout()

plt.savefig(os.path.join(figures_dir, f'images_with_duplicate_tiles.jpg'))

In [ ]:
# mostly black images
img_ids = [
    '03ffa7680.jpg', '8d5521663.jpg', '5a70ef013.jpg', '9a2f9d347.jpg', '37a912dca.jpg', 
    '4add7aa1d.jpg', '3db3ef7cc.jpg', '73fec0637.jpg', '7df214d98.jpg', 'c2955cd21.jpg', 
    'de018b2a8.jpg', '8ce769141.jpg', 'fc0e22a0a.jpg', '770c46cd4.jpg', 'd6e432b79.jpg', 
    'd5d1b6fb8.jpg', '0e4d7dd93.jpg', '9ddeed533.jpg', 'addc11de0.jpg', '65418dfe4.jpg', 
    '119d6a3d6.jpg', '1b287c905.jpg', 'b264b0f96.jpg', '996f92939.jpg', 'e5c3b1f59.jpg']
plot_image_grid(img_ids, 5, 5, filebase='black_images')

In [ ]:
def draw_image_quad(
    img1_id, img2_id, 
    img1_overlap_tag='08', 
    with_scores=True,
    bbox_thickness=4,
    bbox_color=None,
    draw_bboxes=True,
    save=False):
    
    cp2persist(img1_id)
    cp2persist(img2_id)
    
    if not bbox_color:
        if img1_overlap_tag == '08':
            draw_bboxes = False
        if (img1_id, img2_id, img1_overlap_tag) in dup_truth:
            bbox_color = GREEN if dup_truth[(img1_id, img2_id, img1_overlap_tag)] else RED
        else:
            bbox_color = BLUE
            
    fig, ((ax11, ax12), (ax21, ax22)) = plt.subplots(2, 2, figsize=(12, 12))
    
    imgmod1 = ImgMod(os.path.join(samples_dir, img1_id))
    imgmod2 = ImgMod(os.path.join(samples_dir, img2_id))
    
    show_image_pair(ax11, ax12,
                    imgmod1, imgmod2, img1_overlap_tag,
                    draw_bboxes, bbox_thickness, bbox_color,
                    img1_id, img2_id, ticks, median_color_shift=False)
    
    if with_scores:
        scores = get_overlap_pixel_scores(img1_id, img2_id, img1_overlap_tag)
        title1 = f'px0: min: {np.min(scores.px0)}, max: {np.max(scores.px0)}'
        title2 = f'pix: min: {np.min(scores.pix)}, max: {np.max(scores.pix)}'
    else:
        title1 = img1_id
        title2 = img2_id
    
    show_image_pair(ax21, ax22,
                    imgmod1, imgmod2, img1_overlap_tag,
                    draw_bboxes, bbox_thickness, bbox_color,
                    title1, title2, ticks, median_color_shift=True)

    plt.tight_layout()
    
    if save:
        filename = os.path.join(figures_dir, f'{img1_id}_{img2_id}_{img1_overlap_tag}.jpg')
        if os.path.exists(filename):
            print(f'{filename} already exists. Overwriting...')
        else:
            print(f'{filename} saved.')
        plt.savefig(filename)

def draw_image_pair(
    img1_id, img2_id, 
    img1_overlap_tag='08',
    median_color_shift=False,
    with_scores=False,
    bbox_thickness=4,
    bbox_color=None, 
    draw_bboxes=True,
    save=False):
    
    cp2persist(img1_id)
    cp2persist(img2_id)
    
    if not bbox_color:
        if img1_overlap_tag == '08':
            draw_bboxes = False
        if (img1_id, img2_id, img1_overlap_tag) in dup_truth:
            bbox_color = GREEN if dup_truth[(img1_id, img2_id, img1_overlap_tag)] else RED
        else:
            bbox_color = BLUE
    
    if with_scores:
        scores = get_overlap_pixel_scores(img1_id, img2_id, img1_overlap_tag)
        title1 = f'px0: min: {np.min(scores.px0)}, max: {np.max(scores.px0)}'
        title2 = f'pix: min: {np.min(scores.pix)}, max: {np.max(scores.pix)}'
    else:
        title1 = img1_id
        title2 = img2_id
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    
    imgmod1 = ImgMod(os.path.join(samples_dir, img1_id))
    imgmod2 = ImgMod(os.path.join(samples_dir, img2_id))
    
    show_image_pair(ax1, ax2, 
                    imgmod1, imgmod2, img1_overlap_tag, 
                    draw_bboxes, bbox_thickness, bbox_color, 
                    title1, title2, ticks, median_color_shift)
    
    plt.tight_layout()
    
    if save:
        filename = os.path.join(figures_dir, f"{img1_id}_{img2_id}_{img1_overlap_tag}.jpg")
        if os.path.exists(filename):
            print(f"{filename} already exists. Overwriting...")
        else:
            print(f"{filename} saved.")
        plt.savefig(filename)


In [ ]:
draw_image_pair('46b87e21c.jpg', 'f881c203f.jpg', '04', bbox_color='red')

In [ ]:
draw_image_pair('46b87e21c.jpg', 'f881c203f.jpg', '07')

In [ ]:
draw_image_pair('9b34f2f64.jpg', 'e8b058856.jpg', '05', bbox_color='red')

In [ ]:
draw_image_pair('9b34f2f64.jpg', 'e8b058856.jpg', '18')

In [ ]:
draw_image_pair('356f4c539.jpg', '6dd7430f6.jpg', '02')

In [ ]:
draw_image_pair('b8ce38df4.jpg', 'ddfc36407.jpg')

In [ ]:
draw_image_pair('8a0542232.jpg', 'ddfc36407.jpg')

In [ ]:
draw_image_pair('8a0542232.jpg', 'ddfc36407.jpg', '68')

In [ ]:
draw_image_pair('0ef6cd331.jpg', '2095da0cb.jpg')

In [ ]:
draw_image_pair('0efcd3f26.jpg', '89a2baf91.jpg', plot_shift_wrap=False)

In [ ]:
draw_image_pair('2c09a2423.jpg', 'b4eba96e8.jpg', plot_shift_wrap=False)

In [ ]:
draw_image_pair('2556bfc6c.jpg', '2c09a2423.jpg', '28', plot_scores=True)

In [ ]:
draw_image_pair('2556bfc6c.jpg', 'd3474ec95.jpg', plot_shift_wrap=False)

In [ ]:
draw_image_pair('908f11dd6.jpg', 'c5d9bc753.jpg', '02')

In [ ]:
draw_image_pair('2b6c7fd55.jpg', 'c5d9bc753.jpg', '15')

In [ ]:
draw_image_pair('03a5fd8d2.jpg', '676f4cfd0.jpg', plot_scores=True)

In [ ]:
draw_image_pair('536356d11.jpg', '783d9495a.jpg', '15', plot_scores=True)

In [ ]:
draw_image_pair('536356d11.jpg', '88c2acaf8.jpg', '15', plot_scores=True)

In [ ]:
draw_image_pair('861367193.jpg', 'e6e729afa.jpg', '03')

In [ ]:
draw_image_pair('385df9573.jpg', '813a4728e.jpg', '58')

In [ ]:
img_id, pix_thresh = 'd049cb0be.jpg', 5000
img_id, pix_thresh = 'c5d9bc753.jpg', 10000
img_id, pix_thresh = '2556bfc6c.jpg', 5000000
img_id, pix_thresh = 'd3474ec95.jpg', 5000000
pset = set()
overlay_set = set()
for (img1_id, img2_id), overlap_map in overlap_image_maps.items():
    if img_id in (img1_id, img2_id):
        assert len(overlap_map) == 1
        for img1_overlap_tag, scores in overlap_map.items():
#             if np.max(scores.shp) == 0:
#                 break
            if np.max(scores.pix) > pix_thresh:
                break
            print(img1_id, img2_id, img1_overlap_tag)
#             print(scores.pix)
#             print(scores.shp)
        else:
            overlay_set.add((img1_id, img2_id, img1_overlap_tag))
            pset.add(img1_id)
            pset.add(img2_id)

# for p in sorted(pset):
#     print(p)

# plot_image_grid(sorted(pset), 4, 4)

In [ ]:
for i, overlay in enumerate(sorted(overlay_set)):
    pixel_scores = sdcic.gen_pix_scores(*overlay)
    if np.max(pixel_scores) < 5000:
        print((overlay[0], overlay[1]))

In [ ]:
overlap_group1 = [
    '3e98c83f7.jpg', 
    'b356b1f4a.jpg', 
    'd42dcdc8c.jpg', 
    '861367193.jpg', 
    '536356d11.jpg', 
    '9b82e7a76.jpg',
    '783d9495a.jpg', 
    'cc29cb437.jpg',
    '385df9573.jpg']

overlap_group2 = [
    'd0e99b467.jpg', 
    'f0d46bbd8.jpg', 
    'd049cb0be.jpg', 
    '813a4728e.jpg',
    'a4e6f04a8.jpg', 
    'e80ae5e73.jpg', 
    '88c2acaf8.jpg', 
    '30d3278a2.jpg', 
    'e6e729afa.jpg']


In [ ]:
plot_image_grid(overlap_group1, 3, 3)

In [ ]:
plot_image_grid(overlap_group2, 3, 3)

In [ ]:
image_pairs = [(g1, g2) for g1, g2 in zip(overlap_group1, overlap_group2)]

In [ ]:
plot_image_grid(np.array(image_pairs).flatten(), 6, 3)

In [ ]:
pixel_scores = []
for img1_id, img2_id in image_pairs:
    pixel_scores.append(sdcic.gen_pix_scores(img1_id, img2_id, '08'))

print(np.asarray(pixel_scores))

In [ ]:
print(np.asarray(pixel_scores).reshape((9, 3, 3)))

In [ ]:
for img1_id, img2_id in image_pairs:
    draw_wrap_pair(img1_id, img2_id)